In [45]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Manual Modeling
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression





In [46]:
!pip install kagglehub
import os
import kagglehub
path = kagglehub.dataset_download("abdulmalik1518/cars-datasets-2025")

csv_path = os.path.join(path, "Cars Datasets 2025.csv")

df = pd.read_csv(csv_path, encoding="latin1")


df.sample(10)

Defaulting to user installation because normal site-packages is not writeable


,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
659,Tesla,Model S,Dual Electric Motors,100 kWh,670 hp,250 km/h,3.1 sec,"$88,490",Electric,5,967 Nm
35,ASTON MARTIN,DB11,V12,"5,204 cc",600 hp,322 km/h,3.9 sec,"$208,000",Petrol,4,700 Nm
600,Cadillac,CT4-V,2.7L Turbo Inline-4 Gasoline,2700 cc,325 hp,260 km/h,4.9 sec,"$46,595",Petrol,5,500 Nm
869,Kia,Forte GT Turbo,1.6L Turbocharged 4-Cylinder,1591 cc,201 hp,200 km/h,6.5 sec,"$25,500",Petrol,5,264 Nm
963,Acura,ILX Technology Plus,Inline-4,"2,400 cc",201 hp,200 km/h,6.6 sec,"$28,000",Petrol,5,180 Nm
663,Tesla,Model X,Dual Electric Motors (AWD),100 kWh,670 hp,250 km/h,3.8 sec,"$98,490",Electric,7,967 Nm
138,AUDI,Q3,I4,"1,984 cc",248 hp,230 km/h,7.3 sec,"$35,000",Petrol,5,400 Nm
644,Tata Motors,Manza,1.3L Quadrajet Diesel / 1.4L Petrol,1248 cc / 1368 cc,75 hp / 90 hp,160 km/h,14 sec,"$9,000",Diesel/Petrol,5,200 Nm / 115 Nm
453,Nissan,Titan XD,5.6L V8,5552 cc,400 HP,195 km/h,6.9 sec,"$55,000",Petrol,6,560 Nm
729,Chevrolet,Blazer RS,3.6L V6 Gasoline,3600 cc,308 hp,220 km/h,6.6 sec,"$48,900",Petrol,5,366 Nm


In [47]:
df.describe().T

,count,unique,top,freq
Company Names,1218,37,Nissan,149
Cars Names,1218,1201,Macan T,2
Engines,1218,356,I4,64
CC/Battery Capacity,1215,311,"2,000 cc",31
HorsePower,1218,456,355 hp,23
Total Speed,1218,114,250 km/h,145
Performance(0 - 100 )KM/H,1212,180,6.5 sec,45
Cars Prices,1218,535,"$35,000",36
Fuel Types,1218,23,Petrol,871
Seats,1218,19,5,692


In [48]:
df.columns = df.columns.str.strip()
all_screening_cats=['Company Names','Engines','Fuel Types']
df.drop(['Cars Names'], axis=1, inplace=True)


In [49]:
import pandas as pd
import numpy as np

def clean_unit(value):
    if pd.isna(value): return np.nan
    v = str(value).lower()
    for unit in ['cc', 'hp', 'km/h', 'sec', 'nm', ',', '$']:
        v = v.replace(unit, '')
    
    v = v.strip()
    
    try:
        if '-' in v:
            low,high = v.split('-')
            return (float(low) + float(high)) / 2
                
        if '/' in v:
            low, high = v.split('/')
            return (float(low) + float(high)) / 2
        
        return float(v)
    except:
        return np.nan

numeric_cols = [
    'CC/Battery Capacity', 'HorsePower', 'Total Speed', 
    'Performance(0 - 100 )KM/H', 'Cars Prices', 'Seats', 'Torque'
]

for col in numeric_cols:
    df[col] = df[col].apply(clean_unit)
    df[col] = pd.to_numeric(df[col], errors='coerce')



df.head()

,Company Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,V8,3990.0,963.0,340.0,2.5,1100000.0,plug in hyrbrid,2.0,800.0
1,ROLLS ROYCE,V12,6749.0,563.0,250.0,5.3,460000.0,Petrol,5.0,900.0
2,Ford,1.2L Petrol,1200.0,77.5,165.0,10.5,13500.0,Petrol,5.0,120.0
3,MERCEDES,V8,3982.0,630.0,250.0,3.2,161000.0,Petrol,4.0,900.0
4,AUDI,V10,5204.0,602.0,320.0,3.6,253290.0,Petrol,2.0,560.0


In [50]:
import plotly.express as px
df_plot = df.copy()
df_plot['Total Speed'] = df_plot['Total Speed'].fillna(0)


fig1 = px.scatter(
    df_plot,
    x='HorsePower',
    y='Cars Prices',
    color='Company Names',
    size='Total Speed',
    title='HorsePower vs Price',
)
fig1.show()


fig2 = px.histogram(
    df_plot,
    x='Company Names',
    y='Cars Prices',
    color='Company Names',
    title='Price Distribution by Company',
)
fig2.update_layout(showlegend=False)
fig2.show()


corr_matrix = df_plot[['HorsePower', 'Total Speed', 'Cars Prices']].corr()
fig3 = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=['HorsePower', 'Speed', 'Price'],
    y=['HorsePower', 'Speed', 'Price'],
))
fig3.update_layout(title='Correlation Matrix', height=500)
fig3.show()




In [51]:

num_features = ['CC/Battery Capacity', 'HorsePower', 'Total Speed', 'Performance(0 - 100 )KM/H', 'Seats', 'Torque']
cat_features = ['Company Names', 'Fuel Types', 'Engines']

X = df[num_features + cat_features]
y = df['Cars Prices']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


from pycaret.regression import *

train_data = X_train.copy()
train_data['Cars Prices'] = y_train

train_data = train_data.dropna(subset=['Cars Prices'])


s = setup(
    data = train_data, 
    target = 'Cars Prices', 
    session_id = 123,
    categorical_features = ['Company Names', 'Fuel Types', 'Engines'],
    normalize = True, 
    normalize_method = 'zscore', 
    remove_outliers = True,
    imputation_type = 'simple',
    numeric_imputation = 'mean',
    categorical_imputation = 'mode',
    transformation = True,
    verbose = False,
    transform_target=True,
)

best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,54274.0722,147298471168.8058,291932.5248,0.4102,0.3772,0.2103,0.0900
xgboost,Extreme Gradient Boosting,52957.5714,134333498009.6000,287286.8203,0.3967,0.4182,0.2435,0.0640
et,Extra Trees Regressor,54448.5446,150542452063.6344,295733.7005,0.3914,0.3795,0.1979,0.0900
knn,K Neighbors Regressor,58376.3220,153098405408.8000,298915.1771,0.3798,0.4256,0.2399,0.0490
lightgbm,Light Gradient Boosting Machine,56392.3379,149375052478.5310,298290.2640,0.3707,0.4262,0.2449,0.1380
gbr,Gradient Boosting Regressor,56426.0125,168798861525.0482,299532.4488,0.2609,0.3651,0.2255,0.0610
dt,Decision Tree Regressor,61099.7919,151484253805.9726,303960.5792,0.2518,0.4429,0.2805,0.0490
ada,AdaBoost Regressor,65153.7534,158110095990.2190,315986.2211,0.2052,0.4589,0.2685,0.0650
lasso,Lasso Regression,86540.2820,177752509704.4787,346295.3395,-0.0643,0.9637,0.6354,0.3530
en,Elastic Net,86540.2820,177752509704.4787,346295.3395,-0.0643,0.9637,0.6354,0.0470


In [52]:
best_model


RandomForestRegressor(n_jobs=-1, random_state=123)

In [53]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold


X_train_ready = get_config('X_train_transformed')
y_train_ready = get_config('y_train_transformed')

params = {
    'n_estimators': [139],
    'max_depth': [None]
}

cv_strategy = KFold(n_splits=5, shuffle=True, random_state=123)

grid = GridSearchCV(estimator=RandomForestRegressor(random_state=123), 
                    param_grid=params, 
                    cv=cv_strategy, 
                    verbose=1)

grid.fit(X_train_ready, y_train_ready)

print(grid.best_params_)
print("(R2):", grid.best_score_)




Fitting 5 folds for each of 1 candidates, totalling 5 fits
{'max_depth': None, 'n_estimators': 139}
(R2): 0.8797843581225597


In [60]:
y_pred = grid.best_estimator_.predict(X_train_ready)

fig = px.scatter(x=y_train_ready, y=y_pred,labels={'x': 'Actual', 'y': 'Predicted'})

fig.add_scatter(x=[y_train_ready.min(), y_train_ready.max()],
                y=[y_train_ready.min(), y_train_ready.max()],
                mode='lines', line_dash='dash')
fig.show()